<a href="https://colab.research.google.com/github/ShubhamS40/MachineLearning/blob/master/Music_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [8]:
song = pd.read_csv('/content/spotify_millsongdata.csv')


In [9]:
song = song.sample(n=10000, random_state=42)

In [10]:
song

,artist,song,link,text
56679,Wishbone Ash,Right Or Wrong,/w/wishbone+ash/right+or+wrong_20147150.html,Like to have you 'round \r\nWith all the lies...
224,Aerosmith,This Little Light Of Mine,/a/aerosmith/this+little+light+of+mine_2064448...,"This Little Light of Mine (Light of Mine), \r..."
32457,Fall Out Boy,"Dance, Dance",/f/fall+out+boy/dance+dance_10113666.html,She says she's no good with words but I'm wors...
9167,Janis Joplin,Easy Rider,/j/janis+joplin/easy+rider_10147381.html,"Hey mama, mama, come a look at sister, \r\nSh..."
13270,Moody Blues,Peak Hour,/m/moody+blues/peak+hour_20291295.html,I see it all through my window it seems. \r\n...
...,...,...,...,...
18149,Roxette,Hotblooded,/r/roxette/hotblooded_20118732.html,"You know I'm hotblooded, baby... \r\n \r\nGe..."
12895,Michael Bolton,Daddy's Little Girl,/m/michael+bolton/daddys+little+girl_10179384....,"You're the end of the rainbow, my pot of gold ..."
13459,Natalie Cole,Across The Nation,/n/natalie+cole/across+the+nation_20565123.html,Artist: Raw Theme Lyrics \r\nSong: Across The...
4844,Elton John,Between Seventeen And Twenty,/e/elton+john/between+seventeen+twenty_2004617...,I wonder who's sleeping in your sheets tonight...


In [11]:
song.shape

(10000, 4)

In [12]:
song=song.drop(['link'],axis=1).reset_index(drop=True)

In [13]:
song.head()

,artist,song,text
0,Wishbone Ash,Right Or Wrong,Like to have you 'round \r\nWith all the lies...
1,Aerosmith,This Little Light Of Mine,"This Little Light of Mine (Light of Mine), \r..."
2,Fall Out Boy,"Dance, Dance",She says she's no good with words but I'm wors...
3,Janis Joplin,Easy Rider,"Hey mama, mama, come a look at sister, \r\nSh..."
4,Moody Blues,Peak Hour,I see it all through my window it seems. \r\n...


In [14]:
song['tag']=song['artist']+song['text']

In [15]:
song=song.drop(['artist','text'],axis=1).reset_index(drop=True)

In [16]:
song

,song,tag
0,Right Or Wrong,Wishbone AshLike to have you 'round \r\nWith ...
1,This Little Light Of Mine,AerosmithThis Little Light of Mine (Light of M...
2,"Dance, Dance",Fall Out BoyShe says she's no good with words ...
3,Easy Rider,"Janis JoplinHey mama, mama, come a look at sis..."
4,Peak Hour,Moody BluesI see it all through my window it s...
...,...,...
9995,Hotblooded,"RoxetteYou know I'm hotblooded, baby... \r\n ..."
9996,Daddy's Little Girl,"Michael BoltonYou're the end of the rainbow, m..."
9997,Across The Nation,Natalie ColeArtist: Raw Theme Lyrics \r\nSong...
9998,Between Seventeen And Twenty,Elton JohnI wonder who's sleeping in your shee...


In [17]:
import nltk

In [18]:
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
stopwords=stopwords.words('english')

In [21]:
song['tag']=song['tag'].apply(lambda x:x.lower())

In [22]:
song['tag'][0]

"wishbone ashlike to have you 'round  \r\nwith all the lies that you make  \r\nthe things or darkness and you  \r\nsome people say, have just a taste  \r\nright or wrong, you might get burned  \r\nwhat you gain is what you learn  \r\n  \r\ngot one too many women  \r\ndon't know quite which way to go  \r\nthey're all gettin' so expensive  \r\nwhen they walk by themselves  \r\nright or wrong, don't regret  \r\nwhat you went for is what you get  \r\n  \r\nno point in bitter tears  \r\nwhen someone else has cut you down  \r\n'cause there's a time for leavin'  \r\nand there's a time for stickin' around, hey  \r\nright or wrong, you've got to live  \r\nso what you collect is what you give\r\n\r\n"

### Punctuation Removal: All punctuation is removed with a regular expression re.sub(r'[^\w\s]', '', text), keeping only alphanumeric characters and spaces.
Stop Words Removal: Using nltk.corpus.stopwords, stopwords are filtered out from the lyrics.
Lemmatization: Words are reduced to their base form (e.g., "running" becomes "run", "better" becomes "good") using the WordNetLemmatizer.

In [23]:

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data (run these once)
nltk.download('stopwords')
nltk.download('wordnet')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to clean text
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize and remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [24]:
song['tag']=song['tag'].apply(clean_text)

In [25]:
song['tag'][0]

'wishbone ashlike round lie make thing darkness people say taste right wrong might get burned gain learn got one many woman dont know quite way go theyre gettin expensive walk right wrong dont regret went get point bitter tear someone else cut cause there time leavin there time stickin around hey right wrong youve got live collect give'

In [26]:
from sklearn.feature_extraction.text import CountVectorizer

In [27]:
cv=CountVectorizer(max_features=1000,stop_words='english')

In [28]:
vector=cv.fit_transform(song['tag']).toarray()

In [29]:
from sklearn.metrics.pairwise import cosine_similarity

In [30]:

# Get similarity between the first and second document (row 0 and 1)
similarity = cosine_similarity(vector)

print(similarity)  # Output the similarity score between the two documents


[[1.         0.         0.1274118  ... 0.02969569 0.0831411  0.18018749]
 [0.         1.         0.06543076 ... 0.04405514 0.15813365 0.08910591]
 [0.1274118  0.06543076 1.         ... 0.00423761 0.17302156 0.06428243]
 ...
 [0.02969569 0.04405514 0.00423761 ... 1.         0.         0.00898933]
 [0.0831411  0.15813365 0.17302156 ... 0.         1.         0.        ]
 [0.18018749 0.08910591 0.06428243 ... 0.00898933 0.         1.        ]]


In [32]:
similarity[2]

array([0.1274118 , 0.06543076, 1.        , ..., 0.00423761, 0.17302156,
       0.06428243])

In [37]:
def song_recommend(song_name):
    # Get the index of the song with the given song name
    song_index = song[song['song'] == song_name].index[0]

    # Calculate the similarity of the song with all other songs
    distance = similarity[song_index]

    # Sort the songs by similarity and get the top 5 similar songs (excluding the song itself)
    song_list = sorted(list(enumerate(distance)), reverse=True, key=lambda x: x[1])[1:6]

    # Print the names of the recommended songs
    for i in song_list:
        print(song.iloc[i[0]]['song'])

# Example usage



In [39]:
song_recommend('Right Or Wrong')

Nobody's Wrong
Wrong Right Wrong
I've Got A Right To Cry
Where Did We Go Wrong
Wrong Way
